In [1]:
import pandas
import datetime
import numpy
from scipy.optimize import curve_fit
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker
import os
%matplotlib inline

In [10]:
datFileName = "../data/processedSaps-new.txt"
sapsDataDF = pandas.read_csv(datFileName, sep=' ')
# add dst_bins
dstBins = [ -150, -75, -50, -25, -10, 10 ]
sapsDataDF = pandas.concat( [ sapsDataDF, \
                    pandas.cut( sapsDataDF["dst_index"], \
                               bins=dstBins ) ], axis=1 )
sapsDataDF.columns = [ ["dateStr", "hour","sapsLat", \
                     "sapsMLT", "sapsVel", "radId", "poesLat",\
                     "poesMLT", "dst_date", "dst_index", "time", "dst_bin"] ]
# get a csv file with fewer columns for getting grid file data
sapsDataCsv = sapsDataDF[ ["dateStr", "sapsLat", "sapsMLT", "radId", "time"] ]
sapsDataCsv.to_csv("../data/data_for_grid.txt", sep=' ', index=False)
# sapsDataDF["time"] = sapsRawDF["date"].map(lambda x: x.strftime('%H%M'))
sapsDataDF.head()

,dateStr,hour,sapsLat,sapsMLT,sapsVel,radId,poesLat,poesMLT,dst_date,dst_index,time,dst_bin
0,20110107,0,56.5,17.7543,308.2077,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]"
1,20110107,0,55.5,18.0147,224.1588,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]"
2,20110107,0,56.5,17.8749,307.4328,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]"
3,20110107,0,55.5,18.1324,222.4787,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]"
4,20110107,0,56.5,17.9955,305.4201,33.0,62.0082,18.0,2011-01-07 00:00:00,-18.0,0,"(-25, -10]"


In [11]:
grdFileName = "../data/gridVecs.txt"
prcsdGrdFileName = "../data/gridVecs-processed.txt"
# os.system( 'cat ' + grdFileName + ' | tr -s "  " > ' + "../data/gridVecs-processed.txt" )
inpColNames = [ "dateStr", "time", "sapsLat", "sapsLon", \
          "grdVelMagn", "grdVelAzim", "radId", "sapsMLT"]
gridVecsDF = pandas.read_csv(prcsdGrdFileName, sep=' ',\
                             header=None, names=inpColNames)
gridVecsDF.head()

,dateStr,time,sapsLat,sapsLon,grdVelMagn,grdVelAzim,radId,sapsMLT
0,20110107,0,56.5,344.6231,83.6037,-47.4307,33,17.7543
1,20110107,0,55.5,348.5294,96.6077,-42.6488,33,18.0147
2,20110107,0,56.5,346.4322,84.6154,-44.5228,33,17.8749
3,20110107,0,55.5,350.2941,95.6795,-35.1175,33,18.1324
4,20110107,0,56.5,348.2412,84.6160,-39.3513,33,17.9955


In [12]:
# Merge the grid vectors DF and saps data DF
print "saps DF size-->", sapsDataDF.shape
print "grid DF size before merge-->", gridVecsDF.shape
gridVecsDF = pandas.merge( gridVecsDF, sapsDataDF,\
                          on=[ "dateStr", "time", "sapsLat", "radId", "sapsMLT" ] )
print "grid DF size after merge-->", gridVecsDF.shape

saps DF size--> (173136, 12)
grid DF size before merge--> (167653, 8)
grid DF size after merge--> (163444, 15)


In [20]:
# Assuming a perfectly westward flow for SAPS 
# estimate the velocities inside the channel
sapsAzim = -90.
gridVecsDF["estSapsVels"] = gridVecsDF["grdVelMagn"]/ numpy.cos( \
                                numpy.deg2rad( sapsAzim - gridVecsDF["grdVelAzim"] ) )
gridVecsDF.head()

,dateStr,time,sapsLat,sapsLon,grdVelMagn,grdVelAzim,radId,sapsMLT,hour,sapsVel,poesLat,poesMLT,dst_date,dst_index,dst_bin,estSapsVels
0,20110107,0,56.5,344.6231,83.6037,-47.4307,33,17.7543,0,308.2077,62.0082,18.0,2011-01-07 00:00:00,-18.0,"(-25, -10]",113.521085
1,20110107,0,55.5,348.5294,96.6077,-42.6488,33,18.0147,0,224.1588,62.0082,18.0,2011-01-07 00:00:00,-18.0,"(-25, -10]",142.593829
2,20110107,0,56.5,346.4322,84.6154,-44.5228,33,17.8749,0,307.4328,62.0082,18.0,2011-01-07 00:00:00,-18.0,"(-25, -10]",120.673475
3,20110107,0,55.5,350.2941,95.6795,-35.1175,33,18.1324,0,222.4787,62.0082,18.0,2011-01-07 00:00:00,-18.0,"(-25, -10]",166.325336
4,20110107,0,56.5,348.2412,84.6160,-39.3513,33,17.9955,0,305.4201,62.0082,18.0,2011-01-07 00:00:00,-18.0,"(-25, -10]",133.448256
